In [ ]:
import os
import sqlite3
import pandas as pd

isLinux = False
default_linux_path = os.getcwd() + "/work/"
default_windows_path = os.getcwd().replace("\\Data", "\\Documents\\Downloaded\\")
default_path = default_linux_path if isLinux else default_windows_path
DEFAULT_SAVE_DIR = default_path.replace("\\Downloaded", "\\Generated")

### Read dataframe's CSVs

In [ ]:
# read dataframes from CSV
df_references = pd.read_csv(DEFAULT_SAVE_DIR + '\\references_merged.csv') # Source Comma Reference Questionid
df_quiz = pd.read_csv(DEFAULT_SAVE_DIR + '\\quiz_merged.csv') # Index Question Answer1 Answer2 Answer3
df_laws = pd.read_csv(DEFAULT_SAVE_DIR + '\\laws.csv') # Origin Law Article	Comma Content

### Create the database and the tables from dataframes

In [ ]:
config = {
  'user': 'root',
  'password': '',
  'host': 'localhost:3306',
}

conn = sqlite3.connect(**config)
cursor = conn.cursor()

def database_exists(cursor, database_name):
    cursor.execute("SHOW DATABASES;")
    return any(database_name in row for row in cursor)

if not database_exists(cursor, 'antinomic'):
    cursor.execute("CREATE DATABASE antinomic;")
    conn.commit()
    
    # create tables
    cursor.execute('''
    CREATE TABLE references (
        source TEXT,
        comma TEXT,
        reference TEXT,
        question_id INTEGER,
        FOREIGN KEY (question_id) REFERENCES quiz(question_id)
        );
    ''')
    
    cursor.execute('''
    CREATE TABLE quiz (
        question_id INTEGER PRIMARY KEY,
        question TEXT,
        answer1 TEXT,
        answer2 TEXT,
        answer3 TEXT);
        );
    ''')
    
    cursor.execute('''
    CREATE TABLE laws (
        CREATE TABLE laws (
            origin TEXT,
            law TEXT,
            article TEXT,
            comma TEXT,
            content TEXT,
        );
    ''')
    
    # insert data from dataframes
    df_references.to_sql('references', conn, if_exists='replace', index=False)
    df_quiz.to_sql('quiz', conn, if_exists='replace', index=False)
    df_laws.to_sql('laws', conn, if_exists='replace', index=False)
    

cursor.close()
conn.close()